In [3]:
import pandas as pd
import numpy as np
import datetime
import os
import paramiko

In [4]:
host = "107.191.32.220" #hard-coded
port = 22
transport = paramiko.Transport((host, port))

password = "juba-jl1" #hard-coded
username = "jian" #hard-coded

transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)

In [5]:
remote_path="/mnt/drv5/biglots_data/PYD_Segments.zip"
local_path="./"+os.path.basename(remote_path)
sftp.get(remote_path,local_path)
sftp.close()
transport.close()

In [7]:
import zipfile
with zipfile.ZipFile(local_path, 'r') as zip_ref:
    zip_ref.extractall("./")

In [12]:
list_files=os.listdir("./PYD_Segments/")

In [18]:
list_files

['PYD_Active.txt',
 'PYD_Active_Control.txt',
 'PYD_Lapsed.txt',
 'PYD_Lapsed_Control.txt',
 'PYD_Web_Not_Active.txt',
 'PYD_Web_Not_Active_Control.txt']

In [19]:
dict_dfs={}
for file in list_files:
    seg=file.split(".")[0]
    df=pd.read_csv("./PYD_Segments/"+file,dtype=str,sep="\t")
    dict_dfs.update({seg:df})
    print(file,df.shape,df['hashed_customer_num'].nunique(),df['hashed_email_address'].nunique())

PYD_Active.txt (19507524, 2) 19507328 19507265
PYD_Active_Control.txt (199909, 2) 199908 199908
PYD_Lapsed.txt (3282912, 2) 3282901 3282895
PYD_Lapsed_Control.txt (89845, 2) 89844 89844
PYD_Web_Not_Active.txt (1776220, 2) 1776216 1776216
PYD_Web_Not_Active_Control.txt (150000, 2) 150000 150000


In [23]:
list_keys=list(dict_dfs.keys())

for i in range(len(list_keys)):
    k_i=list_keys[i]
    set_id_i=set(dict_dfs[k_i]['hashed_customer_num'].unique().tolist())
    for j in range(i+1,len(list_keys)):
        k_j=list_keys[j]
        set_id_j=set(dict_dfs[k_j]['hashed_customer_num'].unique().tolist())
        print(k_i,k_j,len(set_id_i.intersection(set_id_j)))
        

PYD_Active PYD_Active_Control 0
PYD_Active PYD_Lapsed 0
PYD_Active PYD_Lapsed_Control 0
PYD_Active PYD_Web_Not_Active 59327
PYD_Active PYD_Web_Not_Active_Control 5029
PYD_Active_Control PYD_Lapsed 0
PYD_Active_Control PYD_Lapsed_Control 0
PYD_Active_Control PYD_Web_Not_Active 654
PYD_Active_Control PYD_Web_Not_Active_Control 51
PYD_Lapsed PYD_Lapsed_Control 0
PYD_Lapsed PYD_Web_Not_Active 6
PYD_Lapsed PYD_Web_Not_Active_Control 1
PYD_Lapsed_Control PYD_Web_Not_Active 1
PYD_Lapsed_Control PYD_Web_Not_Active_Control 0
PYD_Web_Not_Active PYD_Web_Not_Active_Control 0


# Uploading

In [24]:
from facebookads.adobjects.customaudience import CustomAudience
from facebookads.adobjects.adaccount import AdAccount
from facebookads.api import FacebookAdsApi
import json
BigLotsAccount_id="act_271491453638620"

jian_APP_ID="2537704939796694"# jian's fb marketing app id
jian_token=json.load(open("/home/jian/Docs/FB_token/lasest_FB_token_jian.json","r"))['latest_FB_token']

In [27]:
def creatfbaudience(AudienceName,AudienceDescription,df,ACCESS_TOKEN,accountid,):
    FacebookAdsApi.init(access_token=ACCESS_TOKEN, api_version='v6.0')
    myAccount = AdAccount(accountid)
    fields = [
    ]
    params = {
      'name': AudienceName,
      'subtype': 'CUSTOM',
      'description': AudienceDescription,
      'customer_file_source': 'PARTNER_PROVIDED_ONLY',
    }
    fbname = myAccount.create_custom_audience(
      fields=fields,
      params=params,
    )['id']
    audience = CustomAudience(fbname)
    users = list(df['hashed_email_address'])
    listlen = len(users)
    chunck_size=50000
    count_uploaded=0
    for i in range(int(np.ceil(listlen/chunck_size))):
        starti = i*50000
        if (i+1)*50000<listlen:
            endi = (i+1)*50000
        else:
            endi = listlen
        list0 = users[starti:endi]
        
        count_uploaded+=len(list0)
        audience.add_users(CustomAudience.Schema.email_hash, list0)
        #print len(list0)   
    print(datetime.datetime.now(),AudienceName,"uploaded",count_uploaded)

In [31]:
for seg,df in dict_dfs.items():
    audience_name = seg
    description="%s_%s_uploaded_on_%s"%(audience_name,df.shape[0],str(datetime.datetime.now().date()))
    # df = pd.read_csv(file,dtype=str)
    creatfbaudience(AudienceName=audience_name,
                    AudienceDescription=description,
                    df=df,
                    ACCESS_TOKEN=jian_token,
                    accountid=BigLotsAccount_id)

2020-06-26 12:12:43.930071 PYD_Active uploaded 19507524
2020-06-26 12:13:01.983596 PYD_Active_Control uploaded 199909
2020-06-26 12:17:11.905290 PYD_Lapsed uploaded 3282912
2020-06-26 12:17:15.948282 PYD_Lapsed_Control uploaded 89845
2020-06-26 12:20:06.739579 PYD_Web_Not_Active uploaded 1776220
2020-06-26 12:20:17.310303 PYD_Web_Not_Active_Control uploaded 150000
